In [1]:
!pip install gmpy2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.3 MB/s eta 0:00:00


In [2]:
import gmpy2
import random
import math
import sys
from gmpy2 import mpz, is_prime

In [3]:
# ===== 配置 =====
LOG_MIN = 70
LOG_MAX = 75
WINDOW_LEN = 2004
TARGET_PRIMES = 12
SAMPLES_PER_DECADE = 1000  # 每个数量级采样 1000 点
RADIUS = 10**6             # 每个中心点搜索 ±1e6
OUTPUT_FILE = "solutions_10_70_to_10_75.txt"

In [4]:
# 预计算小质数用于快速试除（加速 is_prime）
gmpy2.get_context().precision = 200

def fast_presieve(start: mpz) -> bool:
    """快速预筛：估算 [start, start+2003] 中可能的质数上限"""
    survivors = WINDOW_LEN
    # 用前 50 个小质数快速筛
    small_primes = [2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71]
    for p in small_primes:
        first = ((start + p - 1) // p) * p
        if first <= start + WINDOW_LEN - 1:
            count = 1 + (start + WINDOW_LEN - 1 - first) // p
            survivors -= count
            if survivors < TARGET_PRIMES:
                return False
    return True

In [5]:
def count_primes_in_window(start: mpz) -> int:
    """精确计数窗口内质数（使用 gmpy2.is_prime）"""
    count = 0
    for i in range(WINDOW_LEN):
        if is_prime(start + i):
            count += 1
            if count > TARGET_PRIMES:
                break
    return count

In [6]:
def log_uniform_sample(log_min: float, log_max: float) -> mpz:
    """在 [10^log_min, 10^log_max] 内对数均匀采样"""
    log_val = random.uniform(log_min, log_max)
    # 生成 10^log_val 的整数近似
    exponent = int(math.floor(log_val))
    mantissa = 10**(log_val - exponent)
    # 构造整数：mantissa * 10^exponent
    base = mpz(10)**exponent
    return mpz(mantissa * float(base))

In [7]:
def main():
    print(f"🔍 在 [10^{LOG_MIN}, 10^{LOG_MAX}] 范围内搜索解...")
    print(f"每数量级采样 {SAMPLES_PER_DECADE} 点，每点搜索 ±{RADIUS}")
    print(f"结果将写入: {OUTPUT_FILE}")
    
    total_samples = 0
    found_count = 0
    
    with open(OUTPUT_FILE, "a") as f:
        # 按数量级分段采样
        for decade in range(LOG_MIN, LOG_MAX + 1):
            print(f"\n--- 采样 10^{decade} ---")
            for _ in range(SAMPLES_PER_DECADE):
                # 1. 生成中心点
                center_log = random.uniform(decade, decade + 1)
                center = log_uniform_sample(center_log, center_log)
                
                # 2. 在 [center - RADIUS, center + RADIUS] 搜索
                low = center - RADIUS
                if low < mpz(10)**LOG_MIN:
                    low = mpz(10)**LOG_MIN
                
                # 为避免重复，每次只检查一个窗口（随机偏移）
                offset = random.randint(0, 2 * RADIUS - WINDOW_LEN)
                candidate = low + offset
                
                # 3. 快速预筛
                if not fast_presieve(candidate):
                    total_samples += 1
                    continue
                
                # 4. 精确验证
                count = count_primes_in_window(candidate)
                total_samples += 1
                
                if count == TARGET_PRIMES:
                    solution_line = f"N={candidate}\n"
                    f.write(solution_line)
                    f.flush()
                    found_count += 1
                    print(f"\n🎉 找到解 #{found_count}: N = {candidate}")
                    print(f"    科学记数法: ~1e{len(str(candidate))-1}")
                
                # 进度报告
                if total_samples % (SAMPLES_PER_DECADE * 5) == 0:
                    print(f"  已采样 {total_samples} 窗口", end="\r", flush=True)
    
    print(f"\n\n✅ 完成！共采样 {total_samples} 个窗口，找到 {found_count} 个解。")
    print(f"结果保存至: {OUTPUT_FILE}")

In [8]:
if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n🛑 用户中断")
        sys.exit(1)

🔍 在 [10^70, 10^75] 范围内搜索解...
每数量级采样 1000 点，每点搜索 ±1000000
结果将写入: solutions_10_70_to_10_75.txt

--- 采样 10^70 ---

--- 采样 10^71 ---

--- 采样 10^72 ---

--- 采样 10^73 ---

--- 采样 10^74 ---

--- 采样 10^75 ---


✅ 完成！共采样 6000 个窗口，找到 0 个解。
结果保存至: solutions_10_70_to_10_75.txt
